In [1]:
pip install opencv-python cvzone numpy pynput 

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB 9.2 MB/s eta 0:00:05
    --------------------------------------- 0.7/38.8 MB 9.3 MB/s eta 0:00:05
   - -------------------------------------- 1.0/38.8 MB 6.9 MB/s eta 0:00:06
   - -------------------------------------- 1.2/38.8 MB 6.7 MB/s eta 0:00:06
   - -------------------------------------- 1.3/38.8 MB 5.6 MB/s eta 0:00:07
   - -------------------------------------- 1.5/38.8 MB 5.6 MB/s eta 0:00:07
   - -------------------------------------- 1.6/38.8 MB 5.2 MB/s eta 0:00:08
   - -------------------------------------- 1.8/38.8 MB 5.1 MB/s eta 0:00:08
   -- ------------------------------------- 2.0/38.8 MB 4.9 MB/s eta 0:00:08
   -- ------------------------------------- 2.2/38.8 MB 4.7 MB/s eta 0:00:08
   -- ------------------------------------- 2.3/38.8 MB 4.

In [2]:
!pip install mediapipe

   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/50.8 MB 9.2 MB/s eta 0:00:06
   ---------------------------------------- 0.5/50.8 MB 7.0 MB/s eta 0:00:08
    --------------------------------------- 0.9/50.8 MB 7.8 MB/s eta 0:00:07
    --------------------------------------- 1.1/50.8 MB 6.8 MB/s eta 0:00:08
    --------------------------------------- 1.3/50.8 MB 6.2 MB/s eta 0:00:08
   - -------------------------------------- 1.4/50.8 MB 5.4 MB/s eta 0:00:10
   - -------------------------------------- 1.6/50.8 MB 5.3 MB/s eta 0:00:10
   - -------------------------------------- 1.8/50.8 MB 5.2 MB/s eta 0:00:10
   - -------------------------------------- 2.0/50.8 MB 5.0 MB/s eta 0:00:10
   - -------------------------------------- 2.1/50.8 MB 5.0 MB/s eta 0:00:10
   - -------------------------------------- 2.3/50.8 MB 4.7 MB/s eta 0:00:11
   - -------------------------------------- 2.5/50.8 MB 4.7 MB/s eta 0:00:11
   -- 

In [3]:
import cv2
import mediapipe as mp

In [4]:
import cvzone
from cvzone.HandTrackingModule import HandDetector
from time import sleep
import numpy as np
from pynput.keyboard import Controller, Key

In [5]:
# Initialize video capture
cap = cv2.VideoCapture(0)
# Set the frame width and height
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1080)  # Width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 520)   # Height

cv2.namedWindow("Virtual Keyboard", cv2.WINDOW_NORMAL)

# Initialize HandDetector for hand tracking
# Detection and tracking confidence thresholds from CVZone
detector = HandDetector(detectionCon=0.8, minTrackCon=0.2)

# Define virtual keyboard layout
keyboard_keys = [
    ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"],
    ["Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P"],
    ["A", "S", "D", "F", "G", "H", "J", "K", "L", ";"],
    ["Z", "X", "C", "V", "B", "N", "M", ",", ".", "/"],
    ["SPACE", "ENTER", "BACKSPACE"]
]

keyboard = Controller()  # Create a keyboard controller instance

In [8]:
#Drawing buttons
class Button:
    def __init__(self, pos, text, size=(85, 85)):
        self.pos = pos
        self.size = size
        self.text = text
    

In [9]:
def draw_buttons(img, button_list):
    """
    Draws buttons on the given image.

    Args:
        img (numpy.ndarray): The image on which the buttons will be drawn.
        button_list (list): A list of Button objects representing the buttons to be drawn.

    Returns:
        numpy.ndarray: The image with the buttons drawn.
    """
    for button in button_list:
        x, y = button.pos
        w, h = button.size
        cvzone.cornerRect(img, (x, y, w, h), 20, rt=0)
        cv2.rectangle(img, button.pos, (int(x + w), int(y + h)),
                      (37, 238, 250), cv2.FILLED)
        cv2.putText(img, button.text, (x + 20, y + 65),
                    cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 4)
    return img

In [10]:
#Button Objects
button_list = []

# Create Button objects based on keyboard_keys layout
for k in range(len(keyboard_keys)):
    for x, key in enumerate(keyboard_keys[k]):
        if key != "SPACE" and key != "ENTER" and key != "BACKSPACE":
            button_list.append(Button((100 * x + 25, 100 * k + 50), key))
        elif key == "ENTER":
            button_list.append(
                Button((100 * x - 30, 100 * k + 50), key, (220, 85)))
        elif key == "SPACE":
            button_list.append(
                Button((100 * x + 780, 100 * k + 50), key, (220, 85)))
        elif key == "BACKSPACE":
            button_list.append(
                Button((100 * x + 140, 100 * k + 50), key, (400, 85)))

In [11]:
while True:
    success, img = cap.read()  # Read frame from camera
    allHands, img = detector.findHands(img)  # Find hands in the frame

    if len(allHands) == 0:
        lm_list, bbox_info = [], []
    else:
        # Find landmarks and bounding box info
        lm_list, bbox_info = allHands[0]['lmList'], allHands[0]['bbox']

    img = draw_buttons(img, button_list)  # Draw buttons on the frame

    # Check if landmarks (lmList) are detected
    if lm_list:
        for button in button_list:
            x, y = button.pos
            w, h = button.size

            # Check if index finger (lmList[8]) is within the button bounds
            if x < lm_list[8][0] < x + w and y < lm_list[8][1] < y + h:
                cv2.rectangle(img, button.pos, (x + w, y + h),
                              (247, 45, 134), cv2.FILLED) # Highlight the button on hover
                cv2.putText(img, button.text, (x + 20, y + 65),
                            cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 4)

                # Calculate distance between thumb (lmList[4]) and index finger (lmList[8])
                distance = np.sqrt(
                    (lm_list[8][0] - lm_list[4][0])**2 + (lm_list[8][1] - lm_list[4][1])**2)

                # If distance is small, simulate key press
                if distance < 30:
                    # Check for special keys
                    if button.text not in ['ENTER', "BACKSPACE", "SPACE"]:
                        keyboard.press(button.text)  # Press the key
                        # Small delay for better usability & prevent accidental key presses
                        sleep(0.2)
                    else:
                        if button.text == "SPACE":
                            keyboard.press(Key.space)
                            keyboard.release(Key.space)
                            sleep(0.2)

                        elif button.text == "ENTER":
                            keyboard.press(Key.enter)
                            keyboard.release(Key.enter)
                            sleep(0.2)

                        elif button.text == "BACKSPACE":
                            keyboard.press(Key.backspace)
                            keyboard.release(Key.backspace)
                            sleep(0.2)

                        else:
                            pass
                    cv2.rectangle(img, button.pos, (x + w, y + h),
                                  (0, 255, 0), cv2.FILLED)
                    cv2.putText(img, button.text, (x + 20, y + 65),
                                cv2.FONT_HERSHEY_PLAIN, 4, (0, 0, 0), 4)

    # Display the frame with virtual keyboard
    cv2.imshow("Virtual Keyboard", img)
    if cv2.waitKey(1) & 0xFF == 27:  # Exit loop on ESC key press
        break
# Release resources
cap.relecv2.destroyAllWindows()

c:\Users\Yashi\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
